In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
import talib
import pandas_ta as ta
from talib.abstract import *
from math import *
import vectorbt as vbt
import json
%matplotlib inline

In [8]:
df = pd.read_json(r'./freq-user-data/data/binance/BTC_USDT-1m.json')
df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

df['date'] = pd.to_datetime(df['timestamp'], unit='ms', utc=False)
# df.index = df['time']
# df.set_index('time', drop=True, inplace=True)
df.set_index(pd.DatetimeIndex(df["date"]), inplace=True, drop=True)
# df = df[['time', 'symbol', 'source', 'resolution', 'open', 'high', 'low', 'close', 'volume']]
# df.to_csv (r'./data/binance/BTC_USDT-5m.csv', index = None)
# df.set_index('time')


In [9]:
# pd.Timestamp('now').floor('D') + pd.Timedelta(-7, unit='D')

# start_time = datetime(2021,5,1)

start_time = pd.Timestamp('now') + pd.Timedelta(-20, unit='W')
# start_time
ddf = df.loc[df.index >= start_time].copy()
# ddf = df.loc[df['time'] >= start_time]
# ddf
dlen = len(ddf.index)
dlen

199437

In [10]:


start_time = pd.Timestamp('now') + pd.Timedelta(-12, unit='W')
# start_time
ddf = df.loc[df.index >= start_time].copy()

In [11]:
ddf.tail()

,timestamp,open,high,low,close,volume,date
date,,,,,,,
2021-06-22 23:54:00,1624406040000,32402.83,32402.83,32299.99,32319.62,239.117745,2021-06-22 23:54:00
2021-06-22 23:55:00,1624406100000,32319.61,32383.94,32276.24,32364.00,162.277453,2021-06-22 23:55:00
2021-06-22 23:56:00,1624406160000,32363.88,32462.55,32363.88,32445.49,184.730184,2021-06-22 23:56:00
2021-06-22 23:57:00,1624406220000,32445.49,32463.36,32400.00,32455.48,27.366646,2021-06-22 23:57:00
2021-06-22 23:58:00,1624406280000,32455.49,32484.80,32444.69,32461.29,19.574136,2021-06-22 23:58:00


time
2021-03-10 02:30:00             NaN
2021-03-10 02:31:00             NaN
2021-03-10 02:32:00             NaN
2021-03-10 02:33:00             NaN
2021-03-10 02:34:00             NaN
                           ...     
2021-06-22 23:54:00    32347.234698
2021-06-22 23:55:00    32346.618823
2021-06-22 23:56:00    32349.296980
2021-06-22 23:57:00    32352.391200
2021-06-22 23:58:00    32355.694972
Name: (50, close), Length: 150615, dtype: float64

In [15]:
capital = 10000
sl_n2= IntSlider(description="n2", min=0, max=floor(log(dlen, 2.0)), step=1, value=9)
sl_w = IntSlider(description="w", min=0, max=floor(dlen/pow(2,9)), step=1, value=5)
def update_sl_w_range(*args):
    wsize = pow(2,sl_n2.value)
    sl_w.max = floor(dlen/wsize)*2 -1
sl_n2.observe(update_sl_w_range, 'value')


# slCoolOff = IntSlider(description="cool", min=0, step=5, max=600, value=30)
# slDowntrendWait = IntSlider(description="dnt", min=0, max=100, value=30)

slStopLoss = FloatSlider(description="sl", min=0, step=0.05, max=15,value=1.92)
slTrailingStopLoss = FloatSlider(description="ts", min=0, step=0.2, max=15,value=10)
slTakeProfit = FloatSlider(description="tp", min=0, step=0.2, max=15, value=10)

slMa1N = IntSlider(description="ma1n", min=0, step=3, max=400, value=50)
slMa2N = IntSlider(description="ma2n", min=0, step=5, max=600, value=100)
# slMa3N = IntSlider(description="ma3n", min=0, step=20, max=800, value=150)


# slStK = IntSlider(description="stk", min=0, step=1, max=300, value=14)
# slStD = IntSlider(description="std", min=0, step=1, max=200, value=3)
# slStSm = IntSlider(description="stsm", min=0, step=1, max=200, value=2)
# slStT = IntSlider(description="stst", min=0, step=1, max=100, value=20)

slMa1c = FloatSlider(description="ma1c", min=-5, step=0.01, max=5, value=-0.44)
slMa2c = FloatSlider(description="ma2c", min=-5, step=0.01, max=5, value=0.02)
slTrN = IntSlider(description="trn", min=0, step=5, max=800, value=160)
slMrN = IntSlider(description="mrn", min=0, step=5, max=800, value=300)

slXrr = FloatSlider(description="xrr", min=1, step=0.2, max=20, value=2)

ui = widgets.VBox([
    widgets.HBox([sl_n2, sl_w, slTrN, slMrN]),
    widgets.HBox([slMa1N, slMa2N, slMa1c, slMa2c]),
    widgets.HBox([slTrailingStopLoss, slStopLoss, slTakeProfit, slXrr]),
#     widgets.HBox([slStK, slStD, slStSm, slStT])
])

trades = pf = wst = wed = wsz = stock = wdf = False
# iKama1 = Interact.@map kama(&ivdf.close, n=&slKama1N, nfast=&slKama1Fast, nslow=&slKama1Slow)
# iKama1ChannelL = Interact.@map &iKama1 * (1-&slKama1Channel/100) 
# cool, dnt,
idx_ddf = np.arange(len(ddf))
# , stk, std, stsm, stt
def printer(w2log, w, sl, ts, tp, ma1n, ma2n, ma1c, ma2c, trn, mrn, xrr):
#     with open(f'./strat-params.json', "w") as f: f.write(json.dumps({"w2log": sl_n2.value, "w":sl_w.value, "sl":slStopLoss.value, "ts": slTrailingStopLoss.value, 
#            "tp": slTakeProfit.value, "ma1n": slMa1N.value, "ma2n": slMa2N.value, "ma1c": slMa1c.value, "ma2c": slMa2c.value,"trn": slTrN.value })) ;
    
    global wst, wed, wsz, stoch, wdf
    wsz = floor(pow(2,w2log))
    wst = floor(w * wsz / 2)
    wed = floor(wst + wsz)
   
#     vtma1 = vbt.MA.run(ddf['open'], ma1n)
    WMA = vtwma1 = vbt.talib('WMA')
    vtma1 = vbt.MA.run(ddf.close, window=ma1n, ewm=True)
    vtma2 = vbt.MA.run(ddf.close, window=ma2n, ewm=True)
    vtmin = ddf['close'].vbt.rolling_min(mrn)
    vtmax = ddf['close'].vbt.rolling_max(mrn)

    ddf['ma1'] = ma1 = vtma1.ma
    ddf['ma2'] = ma2 = vtma2.ma
    ddf['rmin'] = vtmin
    ddf['rmax'] = vtmax
    ddf['xrr'] = (vtmax - ddf.close) / (ddf.close - vtmin)
    
#     ddf['ma1'] = ma1 = ddf.ta.ema(close=ddf.ta.ohlc4(), length=ma1n)
#     ddf['ma2'] = ma2 = ddf.ta.ema(close=ddf.ta.ohlc4(), length=ma2n)
#     ddf['ma3'] = ma3 = ddf.ta.ema(close=ddf.ta.ohlc4(), length=ma3n)
#     ddf['ma2c'] = ma2*(1+maxc/100)
#     STOCH = vbt.talib('Stoch')
#     stoch = STOCH.run(ddf.high, ddf.low, ddf.close, stk, std, 0, stsm, 0)

#     stoch = ddf.ta.stoch( k=stk, d=std, smooth_k=stsm,append=False); stoch.columns = ['k', 'd'];
    ddf['ma1c'] = ma1 * (1 + ma1c/100)
    ddf['ma2c'] = ma2 * (1 + ma2c/100)

    
    s = ( 
#         stoch.slowk_below(stt, crossover=False) &   
#             stoch.slowd_above(stt, crossover=True) &   
#         stoch.slowk_below(stt, crossover=False) &   
#         stoch.slowk_above(stoch.slowd, crossover=True) &
        (
            (trn < 1) |
            (                
#                 ((ddf['ma1'] - ddf['ma2']).rolling(window=trn).min() > 0) &
                ((ddf['ma1'] - ddf['ma2c']).rolling(window=trn).min() > 0) 
            )
        ) &
#         ((ma2 - ma3).rolling(window=trn).min() > 0) & 
#         (ma1 > ma2) & (ma2 > ma3) &
        (ddf['open'] < ddf['ma2c']) &
        (ddf.xrr > xrr)
    )
    tentries = ddf['entries'] = s.vbt.signals.first()

    
    wdf = ddf[max(ma1n,ma2n):].iloc[wst:wed,:]
    
#     wslowk = stoch.slowk[wdf.index]
#     wslowd = stoch.slowd[wdf.index]

    texits = vbt.OHLCSTEX.run(
        tentries, 
        ddf['open'], 
        ddf['high'], 
        ddf['low'], 
        ddf['close'], 
        sl_stop=sl/100,
        tp_stop=tp/100,
        ts_stop=ts/100
    #     stop_type=None, hit_price=None
    ).exits

    global pf;
    pf = vbt.Portfolio.from_signals(ddf['close'], entries=tentries, exits=texits, direction='longonly', 
                                    init_cash=capital, fees=0.001, slippage=0.0025, freq='1m')

    global trades
    trades = pf.trades.records
    trades['cap'] = trades['pnl'].cumsum() + capital
    ddf_cap = pd.merge(ddf[["date"]], trades[["exit_idx", "cap"]], left_on=idx_ddf, right_index=False, 
             right_on="exit_idx", how="left", sort=False).fillna(method='ffill').set_index('date')

    
    with open(f'./strategy-results/{datetime.now():strat_%Y-%m-%d}.log', "a") as f: 
        f.write(json.dumps(dict({   
            "dt": f'{datetime.now():%Y-%m-%d %HH:%MM:%SS}',
            "period_start": f'{ddf.index[0]}',
            "period_end": f'{ddf.index[-1]}',
            "params": dict({ 'sl': sl, 'ts': ts, 'tp': tp, 'ma1n': ma1n, 'ma2n': ma2n, 'ma1c': ma1c, 'ma2c': ma2c, 'trn': trn, 'mrn': mrn, 'xrr': xrr
#                         'std': std, 'stk': stk, 'stsm': stsm, 'stt': slStT.value
                           }),
            "strategy": "2EMASTH",
            "profit": floor(100* (trades.iloc[-1].cap / capital - 1)) if len(trades.index) else 0. 
        })
    )); f.write("\n");
    
    
    trades_profits = trades[trades['pnl'] > 0]
    trades_losses = trades[trades['pnl'] < 0]    
    
    ddf_entries =  ddf.iloc[trades['entry_idx'].values]
    ddf_profits = ddf.iloc[trades_profits['exit_idx'].values]
    ddf_losses = ddf.iloc[trades_losses['exit_idx'].values]
    
#     fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(16,4), sharex='col', gridspec_kw={'width_ratios': [3,1]})
    fig = plt.figure(constrained_layout=True,figsize=(16,5))
    gs0 = fig.add_gridspec(1, 2, width_ratios=[3,1])

    gs00 = gs0[0,0].subgridspec(2, 1, height_ratios=[4,1])
    gs01 = gs0[0,1].subgridspec(2, 1)

    ax1 = fig.add_subplot(gs00[0,0])
    ax2 = fig.add_subplot(gs00[1,0])

    ax3 = fig.add_subplot(gs01[0,0])
    ax4 = fig.add_subplot(gs01[1,0])


#     wma1 = wdf['ma1']; wma2 = ma2[wdf.index]; wma3 = ma3[wdf.index];
    
    ax1.plot(wdf['close'], lw=0.5)
    ax1.plot(wdf['rmin'], lw=0.5)
    ax1.plot(wdf['rmax'], lw=0.5)
    ax1.plot(wdf['ma1'], lw=0.5)
    ax1.plot(wdf['ma2'], lw=0.5)
    
    ax1.fill_between(wdf.index, wdf['ma1'], wdf['ma1c'][wdf.index], alpha=0.15)
    ax1.fill_between(wdf.index, wdf['ma2'], wdf['ma2c'][wdf.index], alpha=0.15)

    wddf_entries = ddf_entries[ddf_entries.index.isin(wdf.index)]
    wddf_profits = ddf_profits[ddf_profits.index.isin(wdf.index)]
    wddf_losses = ddf_losses[ddf_losses.index.isin(wdf.index)]
    
    wddf_trades = ddf_cap[wst:wed]
    
    
    ax1.vlines(wddf_entries.index,wddf_entries['open'] * 0.97,wddf_entries['open'] * 1.02, lw=0.5, color='blue')
    ax1.vlines(wddf_profits.index,wddf_profits['open'] *0.97,wddf_profits['open'] * 1.02, lw=0.5, color='green')
    ax1.vlines(wddf_losses.index,wddf_losses['open'] * 0.97,wddf_losses['open'] * 1.02, lw=0.5, color='red')
    
    
    ax2.plot(wdf.xrr,lw=0.5,color='green')

#     ax2.plot(wslowk,lw=0.5,color='orange')
#     ax2.plot(wslowd,lw=0.5,color='teal')
#     ax2.axhline(y=20, color='b', linestyle='--',lw=0.5)
#     ax2.axhline(y=80, color='b', linestyle='--',lw=0.5)
#     ax2.vlines(wddf_entries.index,0,100, lw=0.5, color='blue')
#     ax2.vlines(wddf_profits.index,0,100, lw=0.5, color='green')
#     ax2.vlines(wddf_losses.index,0,100, lw=0.5, color='red')

    
    ax3.plot(ddf_cap['cap'])
    ax4.plot(wddf_trades['cap'])
    
    plt.show()
    print(pf.stats())
    print(pf.returns_stats())
    

with open('./strat-params.json') as f: 
    js = json.loads(f.read())
    sl_n2.value=js["w2log"]; sl_w.value=js["w"]; slStopLoss.value=js["sl"]; slTrailingStopLoss.value=js["ts"]; slTrN.value=js["trn"]; slMrN.value=js['mrn'];
    slTakeProfit.value=js["tp"]; slMa1N.value=js["ma1n"]; slMa2N.value=js["ma2n"]; slMa1c.value=js["ma1c"]; slMa2c.value=js["ma2c"]; 
    slXrr.value=js['xrr'];

#     slStK.value=js['stk']; slStD.value=js['std']; slStSm.value=js['stsm']; slStT.value=js['stt'];
    
out = widgets.interactive_output(printer, {"w2log":sl_n2, "w":sl_w, 
                                "sl":slStopLoss, "ts": slTrailingStopLoss, "tp": slTakeProfit, 
                                           "ma1n": slMa1N, "ma2n": slMa2N, "ma1c": slMa1c, "ma2c": slMa2c,"trn": slTrN, 'mrn': slMrN, 'xrr': slXrr,
#                                            "stk": slStK, "std": slStD, "stsm": slStSm, 'stt': slStT
                                          })
    
# interactive_plot = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})
# interactive(printer,w2log=sl_n2, w=sl_w, cool=slCoolOff, dnt=slDowntrendWait,
#                                 sl=slStopLoss, tgt=slTarget);
display(ui, out)

# output = interactive_plot.children[-1]
# # output.layout.width = '1200px'
# output.layout.height = '500px'
# interactive_plot


Output()

In [995]:
with open(f'./strat-params.json', "w") as f: f.write(json.dumps({"w2log": sl_n2.value, "w":sl_w.value, "sl":slStopLoss.value, "ts": slTrailingStopLoss.value, 
           "tp": slTakeProfit.value, "ma1n": slMa1N.value, "ma2n": slMa2N.value, "ma1c": slMa1c.value, "ma2c": slMa2c.value,"trn": slTrN.value, 'mrn': slMrN.value,
             'xrr': slXrr.value})) ;

#     "stk": slStK.value, "std": slStD.value, "stsm": slStSm.value, 'stt': slStT.value,
                    
# with open('./strat-params.json') as f: 
#     js = json.loads(f.read())
#     sl_n2.value=js["w2log"]; sl_w.value=js["w"]; slStopLoss.value=js["sl"]; slTrailingStopLoss.value=js["ts"]; slTrN.value=js["trn"];
#     slTakeProfit.value=js["tp"]; slMa1N.value=js["ma1n"]; slMa2N.value=js["ma2n"]; slMa1c.value=js["ma1c"]; slMa2c.value=js["ma2c"]; 
#     slStK.value=js['stk']; slStD.value=js['std']; slStSm.value=js['stsm'];



In [881]:
# help(ta.stoch)
# floor(100* (trades.iloc[-1].cap / capital - 1)) if len(trades.index) else 0.
wddf

False

In [915]:
talib.EMA(ddf.close, 14)

time
2021-01-31 07:47:00             NaN
2021-01-31 07:48:00             NaN
2021-01-31 07:49:00             NaN
2021-01-31 07:50:00             NaN
2021-01-31 07:51:00             NaN
                           ...     
2021-06-17 00:05:00    38317.308464
2021-06-17 00:06:00    38325.371336
2021-06-17 00:07:00    38341.077824
2021-06-17 00:08:00    38354.692781
2021-06-17 00:09:00    38372.511077
Length: 196220, dtype: float64

In [928]:
, mult, fshift
def apply_ma_func(ts, emalen):
    print(ts)
    ma = talib.EMA(ts[0], emalen)
#     ta.ema(close=ts, length=length)
#     c2ma[:,0] = (ts.close - ma).fshift(fshift)
#     mach[:,0] = ma * mult
    return ma

F = vbt.IndicatorFactory(input_names=['ts'], param_names=['emalen'], output_names=['out'], 
#          in_output_names=['mach, c2ma']
                        )
idf = F.from_apply_func(apply_ma_func).run(ddf.close, 50)
    
# print(F.from_apply_func(apply_func)
#       .run(ddf.close, 50, 1.01,0).ts_out)

[[33622.08]
 [33630.11]
 [33610.06]
 ...
 [38443.17]
 [38443.19]
 [38488.33]]


AssertionError: (1, 1) and (196220, 1) do not match

In [921]:
talib.EMA(ddf.close, 20)

time
2021-01-31 07:47:00             NaN
2021-01-31 07:48:00             NaN
2021-01-31 07:49:00             NaN
2021-01-31 07:50:00             NaN
2021-01-31 07:51:00             NaN
                           ...     
2021-06-17 00:05:00    38308.567234
2021-06-17 00:06:00    38315.158926
2021-06-17 00:07:00    38327.350457
2021-06-17 00:08:00    38338.382794
2021-06-17 00:09:00    38352.663481
Length: 196220, dtype: float64

In [810]:
stoch.k < 20

time
2021-04-11 02:39:00    False
2021-04-11 02:40:00    False
2021-04-11 02:41:00    False
2021-04-11 02:42:00    False
2021-04-11 02:43:00    False
                       ...  
2021-06-17 00:05:00    False
2021-06-17 00:06:00    False
2021-06-17 00:07:00    False
2021-06-17 00:08:00    False
2021-06-17 00:09:00    False
Name: k, Length: 95897, dtype: bool

In [938]:
r = ddf.close.vbt.rolling_min(200)
r

time
2021-02-02 23:20:00         NaN
2021-02-02 23:21:00         NaN
2021-02-02 23:22:00         NaN
2021-02-02 23:23:00         NaN
2021-02-02 23:24:00         NaN
                         ...   
2021-06-17 00:05:00    38168.87
2021-06-17 00:06:00    38168.87
2021-06-17 00:07:00    38168.87
2021-06-17 00:08:00    38168.87
2021-06-17 00:09:00    38168.87
Name: close, Length: 192407, dtype: float64